In [2]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from mbspbs10pc.utils import flatten

In [14]:
item = 86968 # make it work with this one
url = 'http://www9.health.gov.au/mbs/search.cfm?'
payload = {'q': '{}'.format(str(item)),
          'sopt': 'I'}
r = requests.get(url, params=payload)
# r = requests.get('http://www9.health.gov.au/mbs/search.cfm?q={}&sopt=S'.format(str(item)))
soup = BeautifulSoup(r.text, 'html.parser')

In [15]:
type(soup)

bs4.BeautifulSoup

True

In [16]:
print(soup.prettify())

<!--[if HTML5]><![endif]-->
<!DOCTYPE html>
<!--[if lt IE 7 ]><html lang="en-au" class="ie-6"><![endif]-->
<!--[if IE 7 ]><html lang="en-au" class="ie-7"><![endif]-->
<!--[if IE 8 ]><html lang="en-au" class="ie-8"><![endif]-->
<!--[if (gte IE 9)|!(IE)]>
<!-->
<html lang="en-au">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1.0" name="viewport"/>
  <!-- Core CSS -->
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/bootstrap.css/$FILE/bootstrap.no-icons.css?OpenElement&amp;v=1.0" rel="stylesheet"/>
  <link href="http://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet"/>
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/font-awesome.css/$FILE/font-awesome.css" rel="stylesheet"/>
  <link href="http://www.health.gov.au/internet/r/publishing.nsf/AttachmentsByTitle/dha.css/$FILE/dha.css?OpenElement&amp;v=1.0" rel="stylesheet"/>
  <link href="http://www.health

In [13]:
def get_categories(soup):
    """Extract the categories from the soup."""  
    category_elem = filter(lambda x: 'category' in x.text.lower(), soup.find_all('h3'))
    if len(category_elem) > 0:
        category_elem = category_elem[0].text.split('<')[0]
    else:
        category_elem = None
    return category_elem

categories = get_categories(soup)
print('Categories: {}'.format(categories))

Categories: None


In [9]:
def get_info(soup):
    """Extract group, subgroup and subheading from the soup."""
    group = None
    subgroup = None
    subheading = None
    out = []
    for elem in soup.find_all('div'):
        elem_class = elem.get('class')
        if (elem_class is not None) and (u'span9' in elem_class):
            out.append(elem.text)
    group = out[0]
    if len(out) > 1: subgroup = out[1]
    if len(out) > 2: subheading = out[2]
    return group, subgroup, subheading

group, subgroup, subheading = get_info(soup)
print('Group: {}'.format(group))
print('Subgroup: {}'.format(subgroup))
print('Subheading: {}'.format(subheading))

Group: A15 - GP Management Plans, Team Care Arrangements, Multidisciplinary Care Plans
Subgroup: 1 - GP Management Plans, Team Care Arrangements And Multidisciplinary Care Plans
Subheading: None


In [29]:
def get_description(soup):
    """Extract the item description from the soup."""
    description = []
    for elem in soup.find_all('p'):
        if elem.get('align') =='justify':
            description.append(elem.text.encode('utf-8').strip())
    return ' '.join(map(str, description))

description = get_description(soup)
print('Description: {}'.format(description))

Description: Attendance by a medical practitioner (including a general practitioner, but not including a specialist or consultant physician) to review or coordinate a review of: (a) a GP management plan prepared by a medical practitioner (or an associated medical practitioner) to which item 721 applies; or (b) team care arrangements which have been coordinated by the medical practitioner (or an associated medical practitioner) to which item 723 applies


In [14]:
def get_dates(soup):
    """Extract the relevant dates from the soup."""
    dates_keys = []
    dates_values = []
    for elem in soup.find_all('p'):
        if 'date' in elem.text.lower():
            for i, div_elem in enumerate(elem.find_all('div')):
                if u'span8' in div_elem.get('class'):
                    dates_keys.append(div_elem.text)
                elif u'span4' in div_elem.get('class'):
                    dates_values.append(div_elem.text)
    dates = {k: v for k, v in zip(dates_keys, dates_values)}
    return dates

dates = get_dates(soup)
print('Relevant dates: {}'.format(dates))

Relevant dates: {u'Schedule Fee Start Date:': u'01-Nov-2014', u'Description Start Date:': u'01-Nov-2014', u'Item Start Date:': u'01-Nov-2014'}


In [89]:
def get_fees(soup):
    """Extract the fees from the soup."""
    fee = None
    benefit75 = None
    benefit85 = None
    safety_net = None
    for elem in soup.find_all('p'):
        for elem_p in elem.find_all('p'):
            if '$' in elem_p.text:
                splitted = np.array(re.sub(' +', ' ', elem_p.text).split(' '))
                if u'Fee:' in splitted:
                    fees = filter(lambda x: '$' in x, splitted) 
                    fee = np.float(fees[0].split('$')[1])
                    benefit75 = np.float(fees[1].split('$')[1])
                    if len(fees) > 2: benefit85 = np.float(fees[2].split('$')[1]) # this is not always there
                elif u'Safety' in splitted:
                    safety_net = np.float(splitted[-1].split('$')[1])
    return fee, benefit75, benefit85, safety_net

fee, benefit75, benefit85, safety_net = get_fees(soup)

print('Fee: {} $'.format(fee))
print('Benefit 75%: {} $'.format(benefit75))
print('Benefit 85%: {} $'.format(benefit85))
print('Safety net: {} $'.format(safety_net))

Fee: 16.8 $
Benefit 75%: 12.6 $
Benefit 85%: 14.3 $
Safety net: None $


# Test class

In [19]:
from mbspbs10pc import mbs_online
reload(mbs_online);

In [22]:
mbs = mbs_online.MBSOnline(66833)
mbs.set_attributes()

In [23]:
mbs.to_frame()

,66833
Category,Category 6 - PATHOLOGY SERVICES
Group,P2 - Chemical
Subgroup,NaN
Subheading,NaN
Description,"25-hydroxyvitamin D, quantification in serum, ..."
Fee start date,2014-11-01
Description start date,2014-11-01
Item start date,2014-11-01
Fee (A$),30.05
Benefit 75% (A$),22.55


#asd

In [29]:
import pandas as pd
item_map = pd.read_csv('../tmp/item_map.tsv', sep='\t', header=0, index_col=0)

In [31]:
item_map.loc[66833]

MAPPED_ITEM                                                         66833
Item_Start_Date                                                01/11/2014
Item_End_Date                                                  31/12/2999
Item_reuse_flag                                                       NaN
Mapped_Item_Desc        25-hydroxyvitamin d, quantification in serum, ...
Mapped_Item_Category                                                    6
Mapped_Item_Group                                                     P02
Mapped_Item_Subgroup                                                  NaN
CATEGORY_DESC                                          PATHOLOGY SERVICES
GROUP_DESC                                                       CHEMICAL
SUBGROUP_DESC                                                         NaN
BTOS                                                                  502
BTOS_DESC                                                 Pathology Tests
MODIFY_BBI_FLAG                       